!pip install pulp

!pip install Faker

In [1]:
import itertools

In [2]:
from pulp import *
problem = LpProblem("schedule-opt", LpMaximize)

In [3]:
from faker import Faker
fake = Faker() 

In [4]:
from collections import namedtuple

In [5]:
N_l = 1 # number of teachers
N_g = 1 # number of groups
N_s = 2 # number of courses

In [6]:
L = [fake.name() for _ in range(N_l)]
G = range(0, N_g)
types = ['lection', 'lab', 'practice']
subjs = [fake.job() for _ in range(N_s)]
D = range(0, 3)
T = "a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,z,y,z"[0:2*24]

In [7]:
T = T.split(',')[:-1]

In [8]:
def get_all_sp_keys(L, G, types, subjs, D, T):
    return itertools.product(L, G, types, subjs, D, T)

In [9]:
def get_all_gdt_keys(GG, DD, TT):
    return itertools.product(GG, DD, TT)

In [10]:
def get_all_ldt_keys(LL, DD, TT):
    return itertools.product(LL, DD, TT)

In [11]:
SP = LpVariable.dicts("SP", get_all_sp_keys(L, G, types, subjs, D, T), 0, 1, cat=pulp.LpInteger)

In [12]:
Ig = LpVariable.dicts("Ig", get_all_gdt_keys(G, D, T[:-1]), -1, 1, cat=pulp.LpInteger)

In [13]:
Il = LpVariable.dicts("Il", get_all_ldt_keys(L, D, T[:-1]), -1, 1, cat=pulp.LpInteger)

In [14]:
one = LpVariable("one", 1, 1, cat=pulp.LpInteger)

In [15]:
zero = LpVariable("zero", 0, 0, cat=pulp.LpInteger)

In [16]:
def get_x_by_gdt(g, d, t, SP):
    s = lpSum([SP[key] for key in itertools.product(L, [g], types, subjs, [d], [t])])
    return s

In [17]:
def get_x_by_ldt(l, d, t, SP):
    s = lpSum([SP[key] for key in itertools.product([l], G, types, subjs, [d], [t])])
    return s

In [18]:
def sum_x_by_g(g, SP):
    sum = lpSum([SP[key] for key in itertools.product(L, [g], types, subjs, D, T)])
    return sum

In [19]:
def sum_x_by_l(l, SP):
    sum = lpSum([SP[key] for key in itertools.product([l], G, types, subjs, D, T)])
    return sum

In [20]:
alpha = 1.0
beta = 1.0
gamma = 1.0

In [21]:
def get_c(key):
    return 1.0

# Problem

In [22]:
all_lectures = [alpha * get_c(key) * SP[key] for key in get_all_sp_keys(L, G, types, subjs, D, T)]

In [23]:
all_groups_no_gaps = [beta * Ig[key] for key in get_all_gdt_keys(G, D, T[:-1])]

In [24]:
all_teachers_no_gaps = [gamma * Il[key] for key in get_all_ldt_keys(L, D, T[:-1])]

In [25]:
problem += lpSum(all_lectures + all_groups_no_gaps + all_teachers_no_gaps)

# Constraints

## Minimize gaps between lessons for teacher

In [26]:
def next_letter(symbol):
    return T[(T.index(symbol) + 1) % len(T)]

In [27]:
for (teacher, day, time) in itertools.product(L, D, T[:-1]):
    # problem += get_x_by_ldt(teacher, day, time, SP) + get_x_by_ldt(teacher, day, time + 1, SP) -1 >= Il[teacher, day, time]
    problem += get_x_by_ldt(teacher, day, time, SP) + get_x_by_ldt(teacher, day, next_letter(time), SP)- Il[teacher, day, time]>=1

## Minimize gaps between lessons for group

In [28]:
for (group, day, time) in itertools.product(G, D, T[:-1]):
    # problem += get_x_by_gdt(group, day, time, SP) + get_x_by_gdt(group, day, time + 1, SP) -1 >= Ig[group, day, time]
    problem += get_x_by_gdt(group, day, time, SP) + get_x_by_gdt(group, day, next_letter(time), SP)- Ig[group, day, time]>=1

In [29]:
with open('problem.txt', 'w') as f:
    print(problem, file=f)

## Amount of lessons for group must be no more than payload in plan

In [30]:
for g in G:
    problem += sum_x_by_g(g, SP) <= len(D) * 3

## Amount of lessons for teacher must be no more than payload in plan

In [31]:
for teacher in L:
    problem += sum_x_by_l(teacher, SP) <= len(D) * 3

## One lesson per time slot for one group

In [32]:
def sum_x_for_group_at_dt(g, d, t, SP):
    s = 0
    for key in itertools.product(L, [g], types, subjs, [d], [t]):
        s += SP[key]
    return s

In [33]:
for key in get_all_gdt_keys(G, D, T):
    s = sum_x_for_group_at_dt(*key, SP)
    problem += s <= 1
    problem += s >= 0

## One lesson per time slot for one teacher

In [34]:
def sum_x_for_teacher_at_dt(l, d, t, SP):
    s = 0
    for key in itertools.product([l], G, types, subjs, [d], [t]):
        s += SP[key]
    return s

In [35]:
for key in get_all_ldt_keys(L, D, T):
    s = sum_x_for_teacher_at_dt(*key, SP)
    problem += s <= 1
    problem += s >= 0

## No more than 3 lessons for one group per day

In [36]:
def sum_x_for_group_per_day(g, d, SP):
    s = 0
    for key in itertools.product(L, [g], types, subjs, [d], T):
        s += SP[key]
    return s

In [37]:
for (group, day) in itertools.product(G, D):
    problem += sum_x_for_group_per_day(group, day, SP) <= 3

## No more than 4 lessons for one teacher per day

In [38]:
def sum_x_for_teacher_per_day(l, d, SP):
    s = 0
    for key in itertools.product([l], G, types, subjs, [d], T):
        s += SP[key]
    return s

In [39]:
for (teacher, day) in itertools.product(L, D):
    problem += sum_x_for_teacher_per_day(teacher, day, SP) <= 4

## No more than 2 lessons of one subject for group per day

In [40]:
def sum_x_of_same_subject_for_group_per_day(g, d, subj, SP):
    s = 0
    for key in itertools.product(L, [g], types, [subj], [d], T):
        s += SP[key]
    return s

In [41]:
for (group, day, subj) in itertools.product(G, D, subjs):
    problem += sum_x_of_same_subject_for_group_per_day(group, day, subj, SP) <= 2

# Solution

In [42]:
problem.solve(solver=solvers.GLPK(options=['--log', 'tsp.log', '--wmps', 'out.mps']))

1

In [43]:
def print_solution(problem):
    print("Status:", LpStatus[problem.status])
    for v in problem.variables():
         if (v.varValue > 0.0):
            print(v.name, "=", v.varValue)
            #print("Total Cost =", pulp.value(problem.objective))

In [44]:
print_solution(problem)

Status: Optimal
Ig_(0,_0,_'b') = 1
Ig_(0,_0,_'c') = 1
Ig_(0,_1,_'b') = 1
Ig_(0,_1,_'c') = 1
Ig_(0,_2,_'b') = 1
Ig_(0,_2,_'c') = 1
Il_('Robert_Cameron',_0,_'b') = 1
Il_('Robert_Cameron',_0,_'c') = 1
Il_('Robert_Cameron',_1,_'b') = 1
Il_('Robert_Cameron',_1,_'c') = 1
Il_('Robert_Cameron',_2,_'b') = 1
Il_('Robert_Cameron',_2,_'c') = 1
SP_('Robert_Cameron',_0,_'lab',_'Accommodation_manager',_0,_'b') = 1
SP_('Robert_Cameron',_0,_'lab',_'Accommodation_manager',_1,_'b') = 1
SP_('Robert_Cameron',_0,_'lab',_'Accommodation_manager',_2,_'b') = 1
SP_('Robert_Cameron',_0,_'lab',_'Accommodation_manager',_2,_'d') = 1
SP_('Robert_Cameron',_0,_'lab',_'Administrator,_arts',_0,_'c') = 1
SP_('Robert_Cameron',_0,_'lab',_'Administrator,_arts',_0,_'d') = 1
SP_('Robert_Cameron',_0,_'lab',_'Administrator,_arts',_1,_'c') = 1
SP_('Robert_Cameron',_0,_'lab',_'Administrator,_arts',_1,_'d') = 1
SP_('Robert_Cameron',_0,_'lab',_'Administrator,_arts',_2,_'c') = 1


In [45]:
print(value(problem.objective))

-93.0


In [46]:
#print([get_x_by_gdt(0, 0, time, SP) for time in T])
print([SP[('Christopher Terrell', 0, type_,'Electronics engineer', 0, time)] for (type_, time) in itertools.product(types, T)])
#print(SP)

KeyError: ('Christopher Terrell', 0, 'lection', 'Electronics engineer', 0, 'a')